In [13]:
#El individuo perfecto - Práctica 2 - Machine Learning
#Medina Ascencio Carlos Armando

import random
import math

# --- Configuración ---
NUM_GENES = 20
VALOR_MIN = 1
VALOR_MAX = 9
PAREJAS_POR_GRUPO = 10
INDIVIDUOS_POR_GRUPO = PAREJAS_POR_GRUPO * 2
NUM_GRUPOS = 5
MUTACION_PROB = 0.1
FREC_MIGRACION = 5   # cada cuántas generaciones migran
NUM_MIGRANTES = 9    # individuos que migran por subgrupo

# --- Funciones básicas ---
def crear_individuo():
    return [random.randint(VALOR_MIN, VALOR_MAX) for _ in range(NUM_GENES)]

def crear_grupo():
    return [crear_individuo() for _ in range(INDIVIDUOS_POR_GRUPO)]

def cruzar(padre, madre):
    hijo1, hijo2 = [], []
    for p, m in zip(padre, madre):
        prom = (p + m) / 2
        hijo1.append(math.floor(prom))
        hijo2.append(math.ceil(prom))
    return hijo1, hijo2

def mutar(individuo):
    for i in range(len(individuo)):
        if random.random() < MUTACION_PROB:
            cambio = 1 if random.random() < 0.5 else -1
            individuo[i] = min(max(individuo[i] + cambio, VALOR_MIN), VALOR_MAX)
    return individuo

def es_perfecto(individuo):
    return all(g >= 9 for g in individuo)

def fitness(individuo):
    """Medimos cuán cercano está al ser perfecto (suma de genes)."""
    return sum(individuo)

# --- Selección de migrantes ---
def seleccionar_migrantes(grupo):
    migrantes = []
    for _ in range(NUM_MIGRANTES):
        candidatos = random.sample(grupo, 3)  # torneo de 3
        mejor = max(candidatos, key=fitness)
        migrantes.append(mejor[:])  # copia
    return migrantes

# --- Reemplazar peores ---
def reemplazar_peores(grupo, migrantes):
    grupo.sort(key=fitness)  # ordenamos de peor a mejor
    for i in range(len(migrantes)):
        grupo[i] = migrantes[i]  # reemplaza peores
    return grupo

# --- Evolución por grupo ---
def evolucionar_grupo(grupo):
    nueva_gen = []
    random.shuffle(grupo)
    for i in range(0, len(grupo), 2):
        padre, madre = grupo[i], grupo[i+1]
        hijo1, hijo2 = cruzar(padre, madre)
        nueva_gen.append(mutar(hijo1))
        nueva_gen.append(mutar(hijo2))
    return nueva_gen

# --- Evolución total ---
def evolucionar():
    grupos = [crear_grupo() for _ in range(NUM_GRUPOS)]
    generacion = 0

    while True:
        generacion += 1

        # evolucionamos cada grupo de forma independiente
        for g in range(NUM_GRUPOS):
            grupos[g] = evolucionar_grupo(grupos[g])

        # migración cada FREC_MIGRACION generaciones
        if generacion % FREC_MIGRACION == 0:
            migrantes_por_grupo = [seleccionar_migrantes(grupo) for grupo in grupos]
            # rotar migrantes (grupo i manda a grupo (i+1)%NUM_GRUPOS)
            for i in range(NUM_GRUPOS):
                destino = (i + 1) % NUM_GRUPOS
                grupos[destino] = reemplazar_peores(grupos[destino], migrantes_por_grupo[i])

        # evaluar mejor individuo global
        mejor_global = max((ind for grupo in grupos for ind in grupo), key=fitness)

        print(f"Generación {generacion} | Mejor fitness: {fitness(mejor_global)} | Mejor individuo: {mejor_global}")

        if es_perfecto(mejor_global):
            print(f"\n¡Ser perfecto encontrado en la generación {generacion}!")
            print(mejor_global)
            return mejor_global, generacion

# --- Ejecutar ---
if __name__ == "__main__":
    evolucionar()

Generación 1 | Mejor fitness: 119 | Mejor individuo: [7, 8, 8, 7, 7, 7, 5, 6, 5, 5, 5, 5, 5, 5, 5, 4, 8, 6, 5, 6]
Generación 2 | Mejor fitness: 120 | Mejor individuo: [7, 6, 6, 6, 6, 7, 3, 5, 5, 6, 7, 6, 4, 5, 6, 8, 7, 8, 7, 5]
Generación 3 | Mejor fitness: 120 | Mejor individuo: [6, 6, 5, 2, 4, 7, 7, 6, 8, 6, 5, 4, 5, 6, 7, 6, 6, 8, 8, 8]
Generación 4 | Mejor fitness: 123 | Mejor individuo: [7, 6, 6, 4, 5, 6, 5, 7, 7, 6, 5, 5, 5, 5, 9, 7, 6, 8, 7, 7]
Generación 5 | Mejor fitness: 120 | Mejor individuo: [5, 6, 6, 7, 6, 7, 7, 6, 5, 8, 5, 5, 5, 7, 5, 6, 5, 7, 7, 5]
Generación 6 | Mejor fitness: 123 | Mejor individuo: [5, 7, 6, 7, 6, 7, 7, 6, 5, 7, 5, 6, 7, 7, 5, 6, 5, 7, 7, 5]
Generación 7 | Mejor fitness: 119 | Mejor individuo: [6, 6, 6, 7, 6, 7, 5, 7, 6, 5, 5, 5, 6, 7, 6, 5, 6, 6, 6, 6]
Generación 8 | Mejor fitness: 121 | Mejor individuo: [5, 6, 6, 5, 4, 5, 6, 7, 7, 6, 6, 7, 6, 6, 6, 7, 6, 7, 6, 7]
Generación 9 | Mejor fitness: 123 | Mejor individuo: [6, 7, 6, 5, 4, 6, 6, 6, 8, 6, 6, 6